Functions that used with cloud Files

## cloudFiles.schemaLocation
- Stores inferred schema
- Tracks schema evolution
- Makes pipeline restart-safe

In [0]:
map(
    'cloudFiles.schemaLocation',
    '/mnt/schema/customers'
  )

## cloudFiles.inferColumnTypes
- Infers data types instead of string

In [0]:
map(
    'cloudFiles.inferColumnTypes','true'
  )

## cloudFiles.schemaEvolutionMode
- none (fail on schema change)
- addNewColumns (MOST COMMON)
- c) rescue (SAFEST)

In [0]:
 map(
    'cloudFiles.schemaEvolutionMode','none'
  )

In [0]:
 map(
    'cloudFiles.schemaEvolutionMode','addNewColumns'
  )

In [0]:
map(
    'cloudFiles.schemaEvolutionMode','rescue',
    'cloudFiles.rescuedDataColumn','_rescued_data'
  )

## cloudFiles.rescuedDataColumn
- Stores unexpected fields as JSON

## cloudFiles.includeExistingFiles
- Loads historical files once
- Then processes only new files

In [0]:
map(
    'cloudFiles.includeExistingFiles','true'
  )

## cloudFiles.useNotifications
- Uses cloud events instead of file listing
- Faster and cheaper
- cloudFiles.useNotifications is an Auto Loader option that tells Apache Spark Auto Loader to use cloud-native event notifications instead of directory listing to detect new files.

- ❌ Without notifications (default listing)
- Auto Loader:
- Lists files repeatedly
- Scans directories
- Slows down as file count grows
- Costs more on cloud storage APIs
- 
- ✅ With notifications
- Auto Loader:
- Listens to file arrival events
- Knows exactly which files arrived
- Avoids expensive directory scans
- Scales to millions of files

In [0]:
 map(
    'cloudFiles.useNotifications','true'
  )

## cloudFiles.maxFilesPerTrigger
- Controls ingestion rate
- Max 50 files per micro-batch.

In [0]:
 map(
    'cloudFiles.maxFilesPerTrigger','50'
  )

# python tbl properties

In [0]:
@dlt.table(
    name="table_name",
    tblproperties={ ... }
)
-- They control:

-- Optimization
-- Data quality classification
-- Schema behavior
-- Performance
-- Compatibility
-- CDC features

## 1️⃣ Layer / Quality Classification (Very Common)
- "quality": "bronze" | "silver" | "gold"
### What it does
- Logical classification of the table
- Used for governance, monitoring, and readability
- Common in real projects
- 📌 No functional impact, but industry standard

## 2️⃣ Auto Optimization (Highly Recommended)

In [0]:
"pipelines.autoOptimize.managed": "true"


- What it does
- 
- Enables automatic:
- Optimize write
- Auto compaction
- Managed by DLT internally
- 📌 Best for Bronze & Silver

### Delta-level auto optimize (optional)

In [0]:
"delta.autoOptimize.optimizeWrite": "true",
"delta.autoOptimize.autoCompact": "true"

### What it does
- Reduces small files
- Improves query performance
- 📌 Usually handled automatically by DLT, but still valid

##  3️⃣ Schema Evolution & Enforcement

In [0]:
"delta.schema.autoMerge.enabled": "true"

### What it does

- Allows new columns during MERGE / writes
- Useful for evolving schemas
- 📌 Common in Silver

In [0]:
"delta.columnMapping.mode": "name"
-- What it does

-- Enables safe column rename & reorder
-- Prevents corruption during schema evolution
-- 📌 Important for long-running pipelines

## 4️⃣ Append / Mutation Control

In [0]:
"delta.appendOnly": "true"
-- What it does

-- Prevents UPDATE / DELETE
-- Append-only table
-- 📌 Ideal for Bronze ingestion

## 5️⃣ Change Data Capture (CDC)

In [0]:
"delta.enableChangeDataFeed": "true"
-- What it does

-- Enables row-level change tracking
-- Allows incremental downstream consumption
-- 📌 Common in Silver / Gold

## 6️⃣ Deletion & Update Performance (Advanced)


In [0]:
"delta.enableDeletionVectors": "true",
"delta.feature.deletionVectors": "supported",
"delta.minReaderVersion": "3",
"delta.minWriterVersion": "7"
-- What it does

-- Speeds up DELETE / UPDATE / MERGE
-- Avoids rewriting Parquet files
-- 📌 Useful for SCD Type 2, CDC pipelines

## 9️⃣ Performance & Layout

In [0]:
"delta.liquidClustering.enabled": "true"
-- What it does

-- Automatic clustering
-- Reduces need for manual OPTIMIZE/Z-ORDER

- 🔥 Recommended Properties by Layer
- Layer---->	Common Properties
- Bronze--->	quality, appendOnly, autoOptimize
- Silver--->	autoOptimize, schema autoMerge, CDF
- Gold----->	CDF, clustering, constraints